# DL4G - Jass Introduction

In this exercise we will look at some properties of the jass kit environment that can be used to develop your own jass agent.

You will need to have numpy installed, as well as the jass-kit environment.

In [8]:
from jass.game.game_util import *
from jass.game.game_sim import GameSim
from jass.game.game_observation import GameObservation
from jass.game.const import *
from jass.game.rule_schieber import RuleSchieber
from jass.agents.agent import Agent
from jass.agents.agent_random_schieber import AgentRandomSchieber
from jass.arena.arena import Arena


Information about the cards is stored as one-hot encoded arrays, there are several tools available to access the information in the cards. 

Lets deal some random cards first.

In [9]:
# Lets set the seed of the random number generater, so that we get the same results
np.random.seed(1)

# This distributes the cards randomly among the 4 players.
hands = deal_random_hand()
print(hands.shape)

(4, 36)


In [10]:
# There is an entry for each player, to access the cards of the first player
cards = hands[0,:]
print(cards)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0]


In [11]:
# This should be 9 cards
assert(cards.sum() == 9)

# The cards can be converted to other formats for easier reading or processing
print(convert_one_hot_encoded_cards_to_str_encoded_list(cards))

# Each card is encoded as a value between 0 and 35.
print(convert_one_hot_encoded_cards_to_int_encoded_list(cards))


[np.str_('DJ'), np.str_('H6'), np.str_('SK'), np.str_('SJ'), np.str_('S9'), np.str_('CK'), np.str_('CQ'), np.str_('CJ'), np.str_('C7')]
[3, 17, 19, 21, 23, 28, 29, 30, 34]


In [12]:
# There is a method to count colors too
colors = count_colors(cards)
print(colors)

[1 1 3 4]


There is a common jass "rule" to select trump, when you have the "Puur" (Jack of trump) and 3 or more other cards of the same color. 

Task 1: Write a function that returns an array of 4 values that contains a 1 for each color that fulfills the rule or 0 otherwise, i.e. [0 0 0 0] is returned, if you do not have any color with Jack and 3 other cards.


In [13]:
def check_puur_with_four(hand, lowerBorderIndice, upperBorderIndice, puurIndice):
    kindCount = 0
    for i in range (lowerBorderIndice, upperBorderIndice):
        if hand[i] == 1: 
            kindCount+=1
    
    # WICHTIG: kindCount enthält ALLE Karten dieser Farbe (inklusive Jack)
    # Die Regel besagt: Jack + 3 oder mehr ANDERE Karten
    # Also: Wenn Jack vorhanden ist, brauchen wir mindestens 4 Karten total
    # (1 Jack + 3 andere = 4 total)
    
    if hand[puurIndice] == 1 and kindCount >= 4:
        return 1
    else:
        return 0

    

def havePuurWithFour(hand: np.ndarray) -> np.ndarray:
    result = np.zeros(4, dtype=int)
    # Korrekte Kartengrenzen: jede Farbe hat 9 Karten
    puurIndices = [3, 12, 21, 30]
    kind_border_indices = [0, 9, 18, 27, 36]
        
    #split cards of player into for subsets (for each kind one) and check for rule application

    for i in range(4):
        result[i] = check_puur_with_four(hand, kind_border_indices[i], kind_border_indices[i+1], puurIndices[i])
    

    return result

In [14]:
# Debug: Welche Spades Karten haben wir?
print("Alle meine Karten:", convert_one_hot_encoded_cards_to_str_encoded_list(cards))
print()

# Spades Bereich: 18-26
print("Spades Karten (Index 18-26):")
spades_cards = []
for i in range(18, 27):
    if cards[i] == 1:
        # Finde heraus welche Karte das ist
        temp_hand = [0] * 36
        temp_hand[i] = 1
        card_name = convert_one_hot_encoded_cards_to_str_encoded_list(temp_hand)[0]
        spades_cards.append(f"Index {i}: {card_name}")
        print(f"Index {i}: {card_name}")

print(f"\nSpades total: {len(spades_cards)} Karten")
print(f"Spades Jack (Index 21): {'vorhanden' if cards[21] == 1 else 'nicht vorhanden'}")

# Das Problem könnte sein, dass ich falsch zähle oder die falschen Indizes verwende
# Lass mich die Jack-Indizes nochmal überprüfen

# Prüfe ob Indizes 18-26 wirklich alle Spades sind
print("\nAlle Karten im Spades-Bereich (18-26):")
for i in range(18, 27):
    temp_hand = [0] * 36
    temp_hand[i] = 1
    if len(convert_one_hot_encoded_cards_to_str_encoded_list(temp_hand)) > 0:
        card_name = convert_one_hot_encoded_cards_to_str_encoded_list(temp_hand)[0]
        has_card = "✓" if cards[i] == 1 else "✗"
        print(f"Index {i}: {card_name} {has_card}")

result = havePuurWithFour(cards)
print(f"\nErgebnis: {result}")
print("Erwartet: [0, 0, 0, 1]")

Alle meine Karten: [np.str_('DJ'), np.str_('H6'), np.str_('SK'), np.str_('SJ'), np.str_('S9'), np.str_('CK'), np.str_('CQ'), np.str_('CJ'), np.str_('C7')]

Spades Karten (Index 18-26):
Index 19: SK
Index 21: SJ
Index 23: S9

Spades total: 3 Karten
Spades Jack (Index 21): vorhanden

Alle Karten im Spades-Bereich (18-26):
Index 18: SA ✗
Index 19: SK ✓
Index 20: SQ ✗
Index 21: SJ ✓
Index 22: S10 ✗
Index 23: S9 ✓
Index 24: S8 ✗
Index 25: S7 ✗
Index 26: S6 ✗

Ergebnis: [0 0 0 1]
Erwartet: [0, 0, 0, 1]


Another possibility to select trump is by assigning a value to each card, depending on whether the color is trump or not. This table is from the Maturawork of Daniel Graf from 2009: "Jassen auf Basis der Spieltheorie".

In [15]:
# Score for each card of a color from Ace to 6

# score if the color is trump
trump_score = [15, 10, 7, 25, 6, 19, 5, 5, 5]
# score if the color is not trump
no_trump_score = [9, 7, 5, 2, 1, 0, 0, 0, 0]
# score if obenabe is selected (all colors)
obenabe_score = [14, 10, 8, 7, 5, 0, 5, 0, 0,]
# score if uneufe is selected (all colors)
uneufe_score = [0, 2, 1, 1, 5, 5, 7, 9, 11]

Task 2: Implement a function that evaluates a hand that is given as a list of 9 cards and with a given trump value and returns a score depending on the table above. For example the score of our hand ['DJ', 'H6', 'SK', 'SJ', 'S9', 'CK', 'CQ', 'CJ', 'C7'] when Club is trump should be:

2 + 0 + 7 + 2 + 0 + 10 + 7 + 25 + 5 = 58

while the score is 70 if Spade is selected, which is better as you have both the jack and the nine.

You can use the arrays offset_of_card and color_of_card to get the offset (Ace, King, etc.) and color of a card.

In [16]:
def calculate_trump_selection_score(cards, trump: int) -> int:
    # add your code here
    kind_border_indices = [0, 9, 18, 27, 36]
    score = 0
    lower_boundary_trump_kind_indices = kind_border_indices[trump]
    upper_boundary_trump_kind_indices = kind_border_indices[trump + 1]

    for i in range(len(cards)):
        if cards[i] >= lower_boundary_trump_kind_indices and cards[i] < upper_boundary_trump_kind_indices:
            score += trump_score[cards[i] % 9]
        else:
            score += no_trump_score[cards[i] % 9]   
    return score


In [17]:
card_list = convert_one_hot_encoded_cards_to_int_encoded_list(cards)
assert calculate_trump_selection_score(card_list, CLUBS) == 58
assert calculate_trump_selection_score(card_list, SPADES) == 70

## Agents

In order to play a game you have to program an agent that decides on the action. For that you have to override the methods action_trump and action_play_card.

Task 3: Use the function implemented above to select the best trump value. If the calculated trump value is below a threshold (for example let us take 68, as suggested in the work by Daniel Graf) you should "Schiebe", i.e. pass to your partner if you are still allowed to do that.

The game observation allows you to access the information about your card, and if you are the first or second player to select trump.

For playing a card, we just take a random action.

In [18]:
from jass.game import const


class MyAgent(Agent):
    def __init__(self):
        super().__init__()
        # we need a rule object to determine the valid cards
        self._rule = RuleSchieber()
        
    def action_trump(self, obs: GameObservation) -> int:
        """
        Determine trump action for the given observation
        Args:
            obs: the game observation, it must be in a state for trump selection

        Returns:
            selected trump as encoded in jass.game.const or jass.game.const.PUSH
        """
        cardsOfCurrentPlayer = obs.hand
        # convert to int encoded list for easier processing
        cardsOfCurrentPlayer = convert_one_hot_encoded_cards_to_int_encoded_list(cardsOfCurrentPlayer)

        currentMaxTrumpKind = -1
        chosenTrump = -1
        
        #check score for all possible kinds and take max
        possible_kinds = [const.DIAMONDS, const.HEARTS, const.SPADES, const.CLUBS]
        for i in range(len(possible_kinds)):
            trump_score = calculate_trump_selection_score(cardsOfCurrentPlayer, possible_kinds[i])
            if trump_score > currentMaxTrumpKind:
                currentMaxTrumpKind = trump_score
                chosenTrump = possible_kinds[i]
        
        # schiebe if score < 68 if possible, else take max score
        #print("Max trump score: ", currentMaxTrumpKind)
        if obs.forehand == -1:
            if currentMaxTrumpKind < 68:
                return const.PUSH
            else:
                return chosenTrump
        else:
            return chosenTrump   
        

    def action_play_card(self, obs: GameObservation) -> int:
        """
        Determine the card to play.

        Args:
            obs: the game observation

        Returns:
            the card to play, int encoded as defined in jass.game.const
        """
        valid_cards = self._rule.get_valid_cards_from_obs(obs)
        # we use the global random number generator here
        return np.random.choice(np.flatnonzero(valid_cards))
    
    

We can use the game simulation to play a game. We will use that to test our implementation, and then use the arena class to play against other agents

In [19]:
rule = RuleSchieber()
game = GameSim(rule=rule)
agent = MyAgent()

np.random.seed(1)
game.init_from_cards(hands=deal_random_hand(), dealer=NORTH)

In [20]:
obs = game.get_observation()

In [21]:
cards = convert_one_hot_encoded_cards_to_str_encoded_list(obs.hand)
print(cards)
trump = agent.action_trump(obs)
print(trump)
assert trump == HEARTS

[np.str_('DA'), np.str_('DK'), np.str_('D9'), np.str_('D6'), np.str_('HA'), np.str_('HQ'), np.str_('HJ'), np.str_('H8'), np.str_('H7')]
1


In [22]:
# tell the simulation the selected trump
game.action_trump(trump)

In [23]:
# play the game to the end and print the result
while not game.is_done():
    game.action_play_card(agent.action_play_card(game.get_observation()))

print(game.state.points)

[ 10 147]


Another possibility to test agents locally is to use the arena. Let us play 100 games against the Random Agent and see if our trump methods makes any difference.


In [24]:
arena = Arena(nr_games_to_play=100)
arena.set_players(MyAgent(), AgentRandomSchieber(), MyAgent(), AgentRandomSchieber())

In [25]:
arena.play_all_games()

[........................................]  100/ 100 games played


In [26]:
print(arena.points_team_0.sum(), arena.points_team_1.sum())

9026.0 6674.0


Now you can continue with a rule based implemenation of the card play. Also look at the flask implementation of the service to see how you can get your agent online.